In [30]:
import requests
headers = {
        "accept": "application/json"
    }


def  get_concept_url(text):
    url = "https://sali.273v.io/v0.0.1/suggest/concepts"
    params = {
        "text": text,
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data['suggestions'][0]['url']
    else:
        print(f"Request failed with status code {response.status_code}")
        return None
    


def get_sali_tag_from_concept(text,url):
    url = "https://sali.273v.io/v0.0.1"+url
    params = {
        "text": text,
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if len(data['suggestions'])==0:
            return None
        return data['suggestions'][0]["iri"]
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

def get_sali_tag(text):
    url = get_concept_url(text)
    return get_sali_tag_from_concept(text,url)

In [9]:
gpt_tags = {
    "Senior Advocate" : "Mr. V.B.S. Negi",
    "Advocate" : ["Mr. Milind Raj ", "Mr. S.R.S. Gil", "Mr. Lalit Belwal","Mr. Ashish Belwal"],
    "Deputy A.G" : "Mr. Atul Kumar Shah",
    "Brief Holder" : "Mrs. Mamta Joshi",
    "Judge": "Hon’ble Sharad Kumar Sharma, J.",
    "Area of Law" : ["Criminal Law","Indian Penal Code (IPC)"],
    "Asset Type:" : ["Criminal Misc. Application No. 1405 of 2022","Criminal Misc. Application No. 1475 of 2022","Judgment"],
    "Communication Modality":["Legal arguments in court","Court judgment"],
    "Data Format" : "Legal document, court proceedings",
    "Document/Artifact" : "Legal judgment",
    "Engagement Terms":"Legal representation, court proceedings",
    "Event":["Criminal Case No. 6706 of 2020","Court proceedings on 23rd October, 2020"],
    "Forums and Venues": ["High Court of Uttarakhand at Nainital","Court of Judicial Magistrate, 2nd, Haldwani, District Nainital"],
    "Governmental Body"  : "State of Uttarakhand",
    "Industry":"Legal",
    "Location":["High Court of Uttarakhand at Nainital","Haldwani, District Nainital"],
    "Matter Narrative" :"Legal challenge to summoning orders dated 23rd October, 2020, in Criminal Case No. 6706 of 2020.",
    "Objectives": ["Challenge the summoning orders","Quash the summoning orders","Remit the matter back to the Judicial Magistrate for reconsideration"],
    "Service": "Legal representation",
    "Status":"Partly allowed the C482 Applications, quashed the summoning orders, and remitted the matter for reconsideration",
    "Applicants" : ["Minti Bedi and others (Criminal Misc. Application No. 1405 of 2022)","Satkar Singh Bedi (Criminal Misc. Application No. 1475 of 2022)"],
    "Respondents" : "State of Uttarakhand",    
}



In [22]:
from rdflib import Graph, Literal, RDF, URIRef,Namespace, BNode
from rdflib.namespace import FOAF , XSD,OWL,SKOS,RDFS
lmss = Namespace("http://lmss.sali.org/")
# Create a Graph
g = Graph()
# Parse in an RDF file hosted on the Internet
g.parse("main.owl")
class OWLNamedIndividual:

    node_label = OWL.NamedIndividual.n3(namespace_manager=g.namespace_manager).replace(":","_")
    
    def __init__(self, type,label,gpt_tag):
        self.type  = type
        self.label = label
        self.gpt_tag = gpt_tag


named_individuals = []


In [50]:
tag=get_sali_tag("Hon’ble Sharad Kumar Sharma, J. Judge")
print(tag)

http://lmss.sali.org/RCLoy4oZcw6FxnWja39PwnD


In [ ]:
for key in gpt_tags:
    if type(gpt_tags[key])==list:
        for value in gpt_tags[key]:
            neo_tag = get_sali_tag(key+":"+value)
            print(neo_tag)
            print(key+":"+value)
            named_individuals.append(OWLNamedIndividual( neo_tag,value,key))
    else:
        neo_tag = get_sali_tag(key+":"+gpt_tags[key])
        print(neo_tag)
        print(key+":"+gpt_tags[key])
        named_individuals.append(OWLNamedIndividual(neo_tag,gpt_tags[key],key))

In [51]:
#convert to json and store in file
import json
data = []
for individual in named_individuals:
    data.append({
        "type":individual.type,
        "label":individual.label,
        "gpt_tag":individual.gpt_tag
    })
with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [52]:
#load from json file and store in list
named_individuals = []
with open('data.json') as json_file:
    data = json.load(json_file)
    for individual in data:
        named_individuals.append(OWLNamedIndividual(individual["type"],individual["label"],individual["gpt_tag"]))

In [53]:
for individual in named_individuals:
    print(individual.type,individual.label,individual.gpt_tag)
    

http://lmss.sali.org/RcTA2iQmrtqOnX3WrjJIp8 Mr. V.B.S. Negi Senior Advocate
http://lmss.sali.org/RDjeyL29mqIZy1eACYHYZXR Mr. Milind Raj  Advocate
http://lmss.sali.org/RcTA2iQmrtqOnX3WrjJIp8 Mr. S.R.S. Gil Advocate
http://lmss.sali.org/RcTA2iQmrtqOnX3WrjJIp8 Mr. Lalit Belwal Advocate
http://lmss.sali.org/RcTA2iQmrtqOnX3WrjJIp8 Mr. Ashish Belwal Advocate
http://lmss.sali.org/R8k7Nk5pFGXNNV0rDCN83wV Mr. Atul Kumar Shah Deputy A.G
http://lmss.sali.org/RDCkFrEclhipkoHPgckuFY6 Mrs. Mamta Joshi Brief Holder
http://lmss.sali.org/RCLoy4oZcw6FxnWja39PwnD Hon’ble Sharad Kumar Sharma, J. Judge
http://lmss.sali.org/RMZ6lNihK8TG4Flhco1yTy Criminal Law Area of Law
http://lmss.sali.org/RMZ6lNihK8TG4Flhco1yTy Indian Penal Code (IPC) Area of Law
http://lmss.sali.org/R7epmapi47puE4YCEXJUREa Criminal Misc. Application No. 1405 of 2022 Asset Type:
http://lmss.sali.org/R7epmapi47puE4YCEXJUREa Criminal Misc. Application No. 1475 of 2022 Asset Type:
http://lmss.sali.org/R9jpIjzc10qdgl78jXOZtfm Judgment Asset 

In [54]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "Dhananjay@10"

def create_nodes(tx, named_individuals):
    for individual in named_individuals:
        query = (
            "CREATE (:{} {{iri: $iri, label: $label, gpt_tag: $tag}})"
            .format(OWLNamedIndividual.node_label)
        )
        
        
        #print the query which will be executed
        #print(query)
        tx.run(query, iri=individual.type, label=individual.label, tag=individual.gpt_tag)


def create_relationships(tx, owl_classes):
    for owl_class in owl_classes:
        
        for relation in owl_class.relations:
            label = relation.label.replace(":", "_")

            
            query = (
                f"MATCH (a:{OWLNamedIndividual.node_label} {{iri: '{owl_class.iri}'}}), "
                f"(b:{OWLNamedIndividual.node_label} {{iri: '{relation.to_iri}'}}) "
                f"CREATE (a)-[:{label} {{ iri: '{relation.iri}'}} ]->(b)"
            )

            try:
                tx.run(query)
            except Exception as e:
                print(query)




def print_relationship_labels(tx):
    # Your Cypher query
    to ="http://lmss.sali.org/RCLoy4oZcw6FxnWja39PwnD"
    frm = "http://lmss.sali.org/RcTA2iQmrtqOnX3WrjJIp8"
    cypher_query = (
        f"MATCH p=(paa:owl_Class {{ iri: '{frm}' }} )-[:rdfs_subClassOf*]->(any:owl_Class)-[r]->(j:owl_Class {{ iri:'{to}' }})"
        f" WHERE type(r) <> 'rdfs_subClassOf'"
        f" RETURN r;"
    )

    print(cypher_query)
    result = tx.run(cypher_query)
    
    if result.peek() is not None:
        for record in result.records():
            relationship = record["r"]
            label = relationship.type
            print("Relationship Label:", label)
                
        
def delete_all_nodes(tx):
    query = "MATCH (n) DETACH DELETE n"
    tx.run(query)
    



with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
        #session.write_transaction(create_nodes, named_individuals)
        #session.write_transaction(create_relationships, named_individuals)
        #session.write_transaction(delete_all_nodes)
        print_relationship_labels(session)

MATCH p=(paa:owl_Class { iri: 'http://lmss.sali.org/RcTA2iQmrtqOnX3WrjJIp8' } )-[:rdfs_subClassOf*]->(any:owl_Class)-[r]->(j:owl_Class { iri:'http://lmss.sali.org/RCLoy4oZcw6FxnWja39PwnD' }) WHERE type(r) <> 'rdfs_subClassOf' RETURN r;
